# Create New Cycle

This notebook creates a new active cycle:
1. Archives the current active cycle (if any)
2. Creates a new cycle from the template
3. Registers stages and steps in the database

**Note:** Only one cycle can be active at a time.

In [1]:
!pwd

/workspace/workflows/_Tools


In [8]:
# Load helpers
%run /workspace/system/helpers/02_CycleManager.ipynb
%run /workspace/system/helpers/03_StepTracker.ipynb
%run /workspace/system/helpers/04_Display.ipynb

✅ Cycle Manager loaded
✅ Step Tracker loaded
✅ Display utilities loaded


## Step 1: Check Current Status

In [3]:
# Check current cycles
print_header("Current Cycles")

cycles_df = get_cycle_status()
if not cycles_df.empty:
    display_dataframe(cycles_df, max_rows=10)
else:
    print("No cycles found")

# Check for active cycle
active_cycle = get_active_cycle()
if active_cycle:
    print(f"\n⚠️  Active cycle exists: {active_cycle['cycle_name']}")
    print("   This will be archived when creating a new cycle")
else:
    print("\n✅ No active cycle - ready to create new cycle")

 Current Cycles
No cycles found

✅ No active cycle - ready to create new cycle


## Step 2: Enter New Cycle Name

In [4]:
# Get cycle name from user
print_header("Create New Cycle")

# TODO: modify this to use input() for interactive use
cycle_name = "Q1_2024_Analysis"  # Change this to your desired cycle name

print(f"Proposed cycle name: {cycle_name}")

# Validate the name
valid, message = validate_cycle_name(cycle_name)
if valid:
    print(f"✅ {message}")
else:
    print(f"❌ {message}")
    print("\nPlease modify the cycle_name above and run this cell again")

 Create New Cycle
Proposed cycle name: Q1_2024_Analysis
✅ Valid


## Step 3: Create the Cycle

**Warning:** This will archive any existing active cycle!

In [5]:
# Create the new cycle
if valid:
    print_header(f"Creating Cycle: {cycle_name}")
    
    success = create_new_cycle(cycle_name)
    
    if success:
        print("\n" + "=" * 60)
        print("✅ CYCLE CREATED SUCCESSFULLY")
        print("=" * 60)
        
        # Show the new cycle info
        new_cycle = get_active_cycle()
        if new_cycle:
            display_cycle_summary(new_cycle)
        
        # Show directory structure
        active_dir = WORKFLOWS_PATH / f"Active_{cycle_name}"
        if active_dir.exists():
            print(f"\n📁 Active directory: {active_dir}")
            print("\nDirectory structure:")
            for item in sorted(active_dir.iterdir()):
                if item.is_dir():
                    print(f"  📁 {item.name}/")
                    # Show subdirectories for notebooks
                    if item.name == 'notebooks':
                        for subitem in sorted(item.iterdir()):
                            if subitem.is_dir():
                                print(f"      📁 {subitem.name}/")
    else:
        print("\n❌ Failed to create cycle")
else:
    print("\n⚠️  Please fix the cycle name and try again")

 Creating Cycle: Q1_2024_Analysis
📁 Created directory: /workspace/workflows/Active_Q1_2024_Analysis
✅ Registered 1 stages with steps
✅ Cycle 'Q1_2024_Analysis' created successfully

✅ CYCLE CREATED SUCCESSFULLY
 Cycle: Q1_2024_Analysis
+------------+---------------------+
| Property   | Value               |
+============+=====================+
| Status     | active              |
+------------+---------------------+
| Created    | 2025-10-04 01:44:47 |
+------------+---------------------+
| Created By | notebook_user       |
+------------+---------------------+
| Stages     | 0                   |
+------------+---------------------+
| Steps      | 0                   |
+------------+---------------------+

📁 Active directory: /workspace/workflows/Active_Q1_2024_Analysis

Directory structure:
  📁 files/
  📁 logs/
  📁 notebooks/
      📁 Stage_01_Setup/


## Step 4: Verify Creation

In [9]:
# Verify the cycle was created
print_header("Verification")

# Check database
active_cycle = get_active_cycle()
if active_cycle:
    print(f"✅ Active cycle in database: {active_cycle['cycle_name']}")
else:
    print("❌ No active cycle in database")

# Check filesystem
active_dirs = [d for d in WORKFLOWS_PATH.iterdir() if d.is_dir() and d.name.startswith('Active_')]
if active_dirs:
    print(f"✅ Active directory found: {active_dirs[0].name}")
else:
    print("❌ No active directory found")

# Show stages and steps
if active_cycle:
    progress_df = get_current_progress(active_cycle['cycle_name'])
    if not progress_df.empty:
        print(f"\n📊 Registered {len(progress_df)} steps:")
        display_step_progress(progress_df)
    else:
        print("\n⚠️  No steps registered - template may be empty")

 Verification
✅ Active cycle in database: Q1_2024_Analysis
✅ Active directory found: Active_Q1_2024_Analysis

📊 Registered 2 steps:

📁 Stage 1: Setup
  --------------------------------------------------
  🔒 Step 1: Initialize
      Status: ⭕ Not Started
  🔒 Step 2: Validate
      Status: ⭕ Not Started
